# sPyTial Reify Functionality

The `reify` method enables reconstruction of Python objects from CnD data instances, providing the reverse operation of `build_instance`. This allows full round-trip serialization and deserialization, making it possible to:

- **Serialize** Python objects to CnD format for visualization
- **Modify** or analyze the CnD representation 
- **Reconstruct** the original Python objects from the CnD data
- **Extend** the system with custom reifiers for specialized types

This notebook demonstrates the comprehensive reify functionality with interactive examples.

In [1]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

import spytial
from spytial import diagram, CnDDataInstanceBuilder
import json

## Basic Reify Functionality

Let's start with the basic round-trip serialization of common Python data types.

In [2]:
# Create a data instance builder
builder = spytial.CnDDataInstanceBuilder()

# Test data with various types
test_data = {
    "name": "sPyTial Reify Demo",
    "version": 1.0,
    "features": ["visualization", "spatial", "constraints"],
    "settings": {
        "debug": True,
        "max_depth": 100,
        "timeout": None
    },
    "tags": {"python", "library", "research"}
}

print("Original data:")
print(json.dumps(test_data, indent=2, default=str))

Original data:
{
  "name": "sPyTial Reify Demo",
  "version": 1.0,
  "features": [
    "visualization",
    "spatial",
    "constraints"
  ],
  "settings": {
    "debug": true,
    "max_depth": 100,
    "timeout": null
  },
  "tags": "{'research', 'python', 'library'}"
}


In [3]:
# Visualize the original data structure
diagram(test_data)

In [4]:
# Build data instance
data_instance = builder.build_instance(test_data)
print(f"Data instance created with {len(data_instance['atoms'])} atoms")

# Show a sample of the CnD representation
print("\nSample atoms:")
for i, atom in enumerate(data_instance['atoms'][:5]):
    print(f"  {atom}")
if len(data_instance['atoms']) > 5:
    print(f"  ... and {len(data_instance['atoms']) - 5} more")

print(f"\nTotal relations: {len(data_instance['relations'])}")

Data instance created with 15 atoms

Sample atoms:
  {'id': 'n1', 'type': 'str', 'label': 'sPyTial Reify Demo', 'type_hierarchy': ['str', 'object']}
  {'id': 'n2', 'type': 'float', 'label': '1.0', 'type_hierarchy': ['float', 'object']}
  {'id': 'n4', 'type': 'str', 'label': 'visualization', 'type_hierarchy': ['str', 'object']}
  {'id': 'n5', 'type': 'str', 'label': 'spatial', 'type_hierarchy': ['str', 'object']}
  {'id': 'n6', 'type': 'str', 'label': 'constraints', 'type_hierarchy': ['str', 'object']}
  ... and 10 more

Total relations: 12


In [26]:
# Reify back to object
reified_data = builder.reify(data_instance)
print("Reified data:")
print(json.dumps(reified_data, indent=2, default=str))

# Validate round-trip
success = reified_data == test_data
print(f"\n✓ Round-trip successful: {success}")

Reified data:
{
  "project": "sPyTial",
  "metadata": {
    "version": "1.0.0",
    "authors": [
      "Alice",
      "Bob"
    ],
    "license": "MIT"
  },
  "components": [
    {
      "name": "Visualizer",
      "files": [
        "visualizer.py",
        "templates.html"
      ],
      "dependencies": "{'pyyaml', 'jinja2'}"
    },
    {
      "name": "Provider System",
      "files": [
        "provider_system.py"
      ],
      "dependencies": "set()"
    }
  ],
  "tests": {
    "unit": 16,
    "integration": 5,
    "coverage": 95.5
  }
}

✓ Round-trip successful: False


In [6]:
# Visualize the reified data structure
diagram(reified_data)

## Custom Objects Reconstruction

The reify method can also reconstruct custom Python objects with their attributes intact.

In [7]:
class Person:
    def __init__(self, name, age, skills=None):
        self.name = name
        self.age = age
        self.skills = skills or []
    
    def __repr__(self):
        return f"Person(name='{self.name}', age={self.age}, skills={self.skills})"

# Create test objects
original_person = Person("Alice", 30, ["Python", "sPyTial", "Visualization"])
print("Original person:")
print(original_person)

Original person:
Person(name='Alice', age=30, skills=['Python', 'sPyTial', 'Visualization'])


In [8]:
# Visualize the original person object
diagram(original_person)

In [9]:
# Round-trip through reify
data_instance = builder.build_instance(original_person)
reified_person = builder.reify(data_instance)

print("Reified person:")
print(f"Type: {type(reified_person).__name__}")
print(f"Name: {reified_person.name}")
print(f"Age: {reified_person.age}")
print(f"Skills: {reified_person.skills}")

# Validate attributes
success = (reified_person.name == original_person.name and
           reified_person.age == original_person.age and
           reified_person.skills == original_person.skills)
print(f"\n✓ Object reconstruction successful: {success}")

Reified person:
Type: Person
Name: Alice
Age: 30
Skills: ['Python', 'sPyTial', 'Visualization']

✓ Object reconstruction successful: True


In [10]:
# Visualize the reified person object
diagram(reified_person)

## Extensibility with Custom Reifiers

For specialized types that need custom reconstruction logic, you can register custom reifier functions.

In [27]:
class Point:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    
    def __eq__(self, other):
        return isinstance(other, Point) and self.x == other.x and self.y == other.y
    
    def __repr__(self):
        return f"Point({self.x}, {self.y})"

# Custom reifier for Point class
def point_reifier(atom, relations, reify_atom):
    """Custom reifier that properly reconstructs Point objects."""
    point = Point()
    for rel_name, target_ids in relations.items():
        if rel_name == "x" and target_ids:
            point.x = reify_atom(target_ids[0])
        elif rel_name == "y" and target_ids:
            point.y = reify_atom(target_ids[0])
    return point

# Register custom reifier
builder.register_reifier("Point", point_reifier)
print(f"Registered reifiers: {builder.list_reifiers()}")

Registered reifiers: ['Point']


In [12]:
# Test with custom object
original_point = Point(10, 20)
print(f"Original point: {original_point}")

# Visualize original point
diagram(original_point)

Original point: Point(10, 20)


In [13]:
# Round-trip with custom reifier
data_instance = builder.build_instance(original_point)
reified_point = builder.reify(data_instance)

print(f"Reified point: {reified_point}")
print(f"Type: {type(reified_point).__name__}")

success = reified_point == original_point
print(f"\n✓ Custom reifier successful: {success}")

# Visualize reified point
diagram(reified_point)

Reified point: Point(10, 20)
Type: Point

✓ Custom reifier successful: True


## Complex Nested Structures

The reify method handles arbitrarily complex nested data structures with full fidelity.

In [14]:
# Complex nested data structure
complex_data = {
    "project": "sPyTial",
    "metadata": {
        "version": "1.0.0",
        "authors": ["Alice", "Bob"],
        "license": "MIT"
    },
    "components": [
        {
            "name": "Visualizer",
            "files": ["visualizer.py", "templates.html"],
            "dependencies": {"jinja2", "pyyaml"}
        },
        {
            "name": "Provider System", 
            "files": ["provider_system.py"],
            "dependencies": set()
        }
    ],
    "tests": {
        "unit": 16,
        "integration": 5,
        "coverage": 95.5
    }
}

print("Original complex data structure:")
print(f"Project: {complex_data['project']}")
print(f"Version: {complex_data['metadata']['version']}")
print(f"Components: {len(complex_data['components'])}")
print(f"Test coverage: {complex_data['tests']['coverage']}%")

Original complex data structure:
Project: sPyTial
Version: 1.0.0
Components: 2
Test coverage: 95.5%


In [15]:
# Visualize the complex data structure
diagram(complex_data)

In [28]:
# Round-trip through reify
data_instance = builder.build_instance(complex_data)
reified_data = builder.reify(data_instance)

print(f"Data instance: {len(data_instance['atoms'])} atoms, {len(data_instance['relations'])} relations")

print("\nReified complex data structure:")
print(f"Project: {reified_data['project']}")
print(f"Version: {reified_data['metadata']['version']}")
print(f"Components: {len(reified_data['components'])}")
print(f"Test coverage: {reified_data['tests']['coverage']}%")

success = reified_data == complex_data
print(f"\n✓ Complex nested reification successful: {success}")

Data instance: 26 atoms, 16 relations

Reified complex data structure:
Project: sPyTial
Version: 1.0.0
Components: 2
Test coverage: 95.5%

✓ Complex nested reification successful: True


In [17]:
# Visualize the reified complex data structure
diagram(reified_data)

## Interactive Reify Workflow

The following demonstrates an interactive workflow where you can visualize, inspect the CnD representation, and reconstruct objects in the notebook context.

In [18]:
def interactive_reify_workflow(obj, var_name="reified_object"):
    """
    Interactive workflow that visualizes an object, shows its CnD representation,
    and provides the reified object back to the notebook context.
    
    Args:
        obj: Object to visualize and reify
        var_name: Name for the reified object variable
    
    Returns:
        dict: Contains original, data_instance, and reified objects
    """
    print(f"🎯 Interactive Reify Workflow for {type(obj).__name__}")
    print("=" * 50)
    
    # Step 1: Show original
    print("\n📋 STEP 1: Original Object")
    print(f"Type: {type(obj).__name__}")
    print(f"Value: {obj}")
    
    # Step 2: Build CnD representation
    print("\n🔄 STEP 2: Converting to CnD representation...")
    builder = CnDDataInstanceBuilder()
    data_instance = builder.build_instance(obj)
    print(f"✓ Created CnD instance with {len(data_instance['atoms'])} atoms, {len(data_instance['relations'])} relations")
    
    # Step 3: Show visualization
    print("\n🎨 STEP 3: Visualization")
    diagram(obj)
    
    # Step 4: Reify
    print("\n⚡ STEP 4: Reifying back to Python object...")
    reified_obj = builder.reify(data_instance)
    print(f"✓ Reconstructed {type(reified_obj).__name__}")
    
    # Step 5: Validation
    print("\n✅ STEP 5: Validation")
    if reified_obj == obj:
        print("🎉 Perfect round-trip! Objects are identical.")
    else:
        print("⚠️  Objects differ - this may be expected for certain types")
    
    # Step 6: Inject into globals
    print(f"\n📤 STEP 6: Injecting as '{var_name}' into notebook context")
    globals()[var_name] = reified_obj
    print(f"✓ Variable '{var_name}' is now available in this notebook")
    
    result = {
        'original': obj,
        'data_instance': data_instance,
        'reified': reified_obj,
        'builder': builder
    }
    
    print("\n📊 WORKFLOW COMPLETE")
    print(f"Access the reified object via: {var_name}")
    print("Access full workflow data via the returned dictionary")
    
    return result

### Interactive Demo: Simple Data Structure

In [19]:
# Try the interactive workflow with a simple data structure
demo_data = {
    "name": "Interactive Demo",
    "items": [1, 2, 3, 4],
    "nested": {"a": 10, "b": 20}
}

workflow_result = interactive_reify_workflow(demo_data, "my_reified_data")

🎯 Interactive Reify Workflow for dict

📋 STEP 1: Original Object
Type: dict
Value: {'name': 'Interactive Demo', 'items': [1, 2, 3, 4], 'nested': {'a': 10, 'b': 20}}

🔄 STEP 2: Converting to CnD representation...
✓ Created CnD instance with 10 atoms, 9 relations

🎨 STEP 3: Visualization



⚡ STEP 4: Reifying back to Python object...
✓ Reconstructed dict

✅ STEP 5: Validation
🎉 Perfect round-trip! Objects are identical.

📤 STEP 6: Injecting as 'my_reified_data' into notebook context
✓ Variable 'my_reified_data' is now available in this notebook

📊 WORKFLOW COMPLETE
Access the reified object via: my_reified_data
Access full workflow data via the returned dictionary


In [20]:
# Now you can access the reified object directly
print("Accessing the reified object:")
print(f"my_reified_data = {my_reified_data}")
print(f"Type: {type(my_reified_data)}")
print(f"my_reified_data['items'] = {my_reified_data['items']}")

Accessing the reified object:
my_reified_data = {'name': 'Interactive Demo', 'items': [1, 2, 3, 4], 'nested': {'a': 10, 'b': 20}}
Type: <class 'dict'>
my_reified_data['items'] = [1, 2, 3, 4]


### Interactive Demo: Custom Object

In [21]:
# Try with a custom object
demo_person = Person("Interactive User", 25, ["Python", "Jupyter", "sPyTial"])
workflow_result2 = interactive_reify_workflow(demo_person, "my_reified_person")

🎯 Interactive Reify Workflow for Person

📋 STEP 1: Original Object
Type: Person
Value: Person(name='Interactive User', age=25, skills=['Python', 'Jupyter', 'sPyTial'])

🔄 STEP 2: Converting to CnD representation...
✓ Created CnD instance with 7 atoms, 6 relations

🎨 STEP 3: Visualization



⚡ STEP 4: Reifying back to Python object...
✓ Reconstructed Person

✅ STEP 5: Validation
⚠️  Objects differ - this may be expected for certain types

📤 STEP 6: Injecting as 'my_reified_person' into notebook context
✓ Variable 'my_reified_person' is now available in this notebook

📊 WORKFLOW COMPLETE
Access the reified object via: my_reified_person
Access full workflow data via the returned dictionary


In [22]:
# Access the reified person object
print("Accessing the reified person:")
print(f"my_reified_person = {my_reified_person}")
print(f"Name: {my_reified_person.name}")
print(f"Skills: {my_reified_person.skills}")

# You can modify the reified object
my_reified_person.skills.append("Reification")
print(f"\nAfter modification: {my_reified_person.skills}")

Accessing the reified person:
my_reified_person = Person(name='Interactive User', age=25, skills=['Python', 'Jupyter', 'sPyTial'])
Name: Interactive User
Skills: ['Python', 'Jupyter', 'sPyTial']

After modification: ['Python', 'Jupyter', 'sPyTial', 'Reification']


## Text-Based Object Insertion

For cases where you want to examine or recreate the object as code, here's a text-based approach:

In [23]:
def reify_as_text(obj, var_name="reconstructed"):
    """
    Generate text representation that can be used to recreate the reified object.
    """
    builder = CnDDataInstanceBuilder()
    data_instance = builder.build_instance(obj)
    reified_obj = builder.reify(data_instance)
    
    # Generate text representation based on type
    if isinstance(reified_obj, dict):
        text_repr = f"{var_name} = {repr(reified_obj)}"
    elif isinstance(reified_obj, (list, tuple, set)):
        text_repr = f"{var_name} = {repr(reified_obj)}"
    elif hasattr(reified_obj, '__dict__'):
        # For custom objects, show attribute assignment
        class_name = type(reified_obj).__name__
        text_repr = f"# Reconstructed {class_name} object\n"
        text_repr += f"{var_name} = {class_name}()\n"
        for attr, value in reified_obj.__dict__.items():
            text_repr += f"{var_name}.{attr} = {repr(value)}\n"
    else:
        text_repr = f"{var_name} = {repr(reified_obj)}"
    
    print("📝 Generated reconstruction code:")
    print("-" * 40)
    print(text_repr)
    print("-" * 40)
    
    return text_repr, reified_obj

In [24]:
# Demo text-based reconstruction
sample_obj = {"config": {"debug": True, "workers": 4}, "data": ["a", "b", "c"]}
text_code, reified = reify_as_text(sample_obj, "my_config")

print("\nYou can copy and paste the above code to recreate the object.")

📝 Generated reconstruction code:
----------------------------------------
my_config = {'config': {'debug': True, 'workers': 4}, 'data': ['a', 'b', 'c']}
----------------------------------------

You can copy and paste the above code to recreate the object.


In [25]:
# Demo with custom object
sample_person = Person("Text Demo", 28, ["Documentation", "Testing"])
text_code2, reified2 = reify_as_text(sample_person, "recreated_person")

📝 Generated reconstruction code:
----------------------------------------
# Reconstructed Person object
recreated_person = Person()
recreated_person.name = 'Text Demo'
recreated_person.age = 28
recreated_person.skills = ['Documentation', 'Testing']

----------------------------------------


## Summary

The sPyTial reify functionality provides:

✅ **Full Round-trip Serialization**: Convert Python objects to CnD format and back with perfect fidelity

✅ **Comprehensive Type Support**: Primitives, collections, custom objects, and nested structures

✅ **Extensibility**: Register custom reifiers for specialized reconstruction logic

✅ **Interactive Workflows**: Visualize, inspect, and reconstruct objects in notebook context

✅ **Text-based Reconstruction**: Generate code to recreate objects programmatically

This enables powerful workflows for data analysis, debugging, and interactive exploration of complex Python objects through spatial visualization.